In [2]:
import cv2
import numpy as np


In [3]:

img = cv2.imread("test3.png")

src = np.array([
    [320, 150],
    [700, 180],
    [750, 480],
    [280, 500]
], dtype=np.float32)

w, h = 400, 300
dst = np.array([
    [0, 0],
    [w-1, 0],
    [w-1, h-1],
    [0, h-1]
], dtype=np.float32)


In [ ]:

def get_h(s, d):
    m = []
    for i in range(4):
        x, y = s[i]
        u, v = d[i]
        m.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        m.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])
    m = np.array(m, dtype=np.float64)
    _, _, vt = np.linalg.svd(m)
    h = vt[-1] / vt[-1, -1]
    return h.reshape(3, 3)

H = get_h(src, dst)
Hinv = np.linalg.inv(H)
out = np.zeros((h, w, 3), dtype=np.uint8)

for yy in range(h):
    for xx in range(w):
        v = np.array([xx, yy, 1], dtype=np.float64)
        p = Hinv @ v
        p /= p[2]
        sx, sy = p[0], p[1]
        if 0 <= sx < img.shape[1]-1 and 0 <= sy < img.shape[0]-1:
            x0, y0 = int(sx), int(sy)
            dx, dy = sx - x0, sy - y0
            c00 = img[y0, x0]
            c10 = img[y0, x0+1]
            c01 = img[y0+1, x0]
            c11 = img[y0+1, x0+1]
            top = (1-dx)*c00 + dx*c10
            bottom = (1-dx)*c01 + dx*c11
            out[yy, xx] = (1-dy)*top + dy*bottom

cv2.imshow("orig", img)
cv2.imshow("fixed", out)
cv2.waitKey(0)
cv2.destroyAllWindows()
